# Libabry

In [2]:
import os
import pandas as pd
import re
import emoji

from bs4 import BeautifulSoup
from textblob import TextBlob

# Data Load

In [3]:
path = os.path.join(".", "Data", "archive", "CommentsJan2017.csv")
TestDF = pd.read_csv(path, header=0)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_24540\1265303922.py:2: DtypeWarning: Columns (14,15,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  TestDF = pd.read_csv(path, header=0)


In [4]:
base_path = os.path.join(".", "Data", "archive")

filenames = [
    "CommentsJan2017.csv", "CommentsFeb2017.csv", "CommentsMarch2017.csv",
    "CommentsApril2017.csv", "CommentsMay2017.csv",
    "CommentsJan2018.csv", "CommentsFeb2018.csv", "CommentsMarch2018.csv",
    "CommentsApril2018.csv"
]

# Read and store all DataFrames
all_comments = []
for fname in filenames:
    path = os.path.join(base_path, fname)
    df = pd.read_csv(path, header=0)
    all_comments.append(df)

# Concatenate all into a single DataFrame
df = pd.concat(all_comments, ignore_index=True)
df = df.reset_index()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_24540\2599382123.py:14: DtypeWarning: Columns (14,15,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, header=0)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_24540\2599382123.py:14: DtypeWarning: Columns (14,15,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, header=0)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_24540\2599382123.py:14: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, header=0)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_24540\2599382123.py:14: DtypeWarning: Columns (25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, header=0)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_24540\2599382123.py:14: DtypeWarning: Columns (25,26) have mixed types. Specify dtype option on import or set low_memory=

## Dataframe information

In [5]:
TestDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231449 entries, 0 to 231448
Data columns (total 34 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   approveDate            231449 non-null  int64  
 1   articleID              231449 non-null  object 
 2   articleWordCount       231449 non-null  int64  
 3   commentBody            231449 non-null  object 
 4   commentID              231449 non-null  float64
 5   commentSequence        231449 non-null  float64
 6   commentTitle           218042 non-null  object 
 7   commentType            231449 non-null  object 
 8   createDate             231449 non-null  int64  
 9   depth                  231449 non-null  float64
 10  editorsSelection       231449 non-null  int64  
 11  inReplyTo              231449 non-null  int64  
 12  newDesk                231449 non-null  object 
 13  parentID               231449 non-null  float64
 14  parentUserDisplayName  56226 non-nul

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2176364 entries, 0 to 2176363
Data columns (total 35 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   index                  int64  
 1   approveDate            int64  
 2   articleID              object 
 3   articleWordCount       float64
 4   commentBody            object 
 5   commentID              float64
 6   commentSequence        float64
 7   commentTitle           object 
 8   commentType            object 
 9   createDate             float64
 10  depth                  float64
 11  editorsSelection       int64  
 12  inReplyTo              float64
 13  newDesk                object 
 14  parentID               float64
 15  parentUserDisplayName  object 
 16  permID                 object 
 17  picURL                 object 
 18  printPage              float64
 19  recommendations        float64
 20  recommendedFlag        float64
 21  replyCount             float64
 22  reportAbuseFlag   

# Preprocessing

In [7]:
TestDF['commentBody'] = TestDF['commentBody'].astype(str)
TestDF.head(5)

,approveDate,articleID,articleWordCount,commentBody,commentID,commentSequence,commentTitle,commentType,createDate,depth,...,status,timespeople,trusted,updateDate,userDisplayName,userID,userLocation,userTitle,userURL,typeOfMaterial
0,1483455908,58691a5795d0e039260788b9,1324,For all you Americans out there --- still rejo...,20969730.0,20969730.0,<br/>,comment,1483426105,1.0,...,approved,1,0,1483455908,N. Smith,64679318.0,New York City,NaN,NaN,News
1,1483455656,58691a5795d0e039260788b9,1324,Obamas policies may prove to be the least of t...,20969325.0,20969325.0,<br/>,comment,1483417407,1.0,...,approved,1,0,1483455656,Kilocharlie,69254188.0,Phoenix,NaN,NaN,News
2,1483455655,58691a5795d0e039260788b9,1324,Democrats are comprised of malcontents who gen...,20969855.0,20969855.0,<br/>,comment,1483431433,1.0,...,approved,1,0,1483455655,Frank Fryer,76788711.0,Florida,NaN,NaN,News
3,1483455653,58691a5795d0e039260788b9,1324,The picture in this article is the face of con...,20969407.0,20969407.0,<br/>,comment,1483418853,1.0,...,approved,1,0,1483455653,James Young,72718862.0,Seattle,NaN,NaN,News
4,1483455216,58691a5795d0e039260788b9,1324,Elections have consequences.,20969274.0,20969274.0,NaN,comment,1483416766,1.0,...,approved,1,0,1483455216,M.,7529267.0,Seattle,NaN,NaN,News


In [8]:
words_map = {
    "u": "you",
    "ur": "your",
    "pls": "please",
    "plz": "please",
    "idk": "I don't know",
    "im": "I am",
    "lol": "laugh out loud",
    "lmao": "laughing my ass off",
    "wtf": "what the heck",
    "fyi": "for your information",
    "diy": "do it yourself",
    "omg": "oh my god",
    "btw": "by the way",
    "tbh": "to be honest",
    "ded": "dead"
}

# Blob correction
def blob_correct(text):
    if not isinstance(text, str):
        return ""
    
    corrected_word = []

    for word in text.split():
        if len(word) <= 4:
            word = str(TextBlob(word).correct())
        corrected_word.append(word)
    return " ".join(corrected_word)
        
def clean_comment(text):
    if not isinstance(text, str):
        return ""
    
    #Remove html markup
    text = BeautifulSoup(text, "html.parser").get_text()
    # Lower case
    text = text.lower()

    # Normalize emoji
    text = emoji.demojize(text)

    # Remove URL of all kind
    text = re.sub(r"http:\S+|www\S+", "", text)

    # Remove mention and hashtag
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#", "", text)

    # Handle acronym words first phase
    words = text.split()
    words = [words_map.get(word, word) for word in words]
    text = " ".join(words)

    #Correct small typo
    text = blob_correct(text)

    # Normalize repeated character. Ex: yeeeees -> yees
    text = re.sub(r"(.)\1{2,}", r"\1", text)

    # Remove non-alphanumeric except emoji alias
    text = re.sub(r"[^a-zA-Z0-9\s_:]", "", text)
    
    # Clean up extra whitespace
    text = re.sub(r"\s+", " ", text).strip()

    #textblob = TextBlob(text)
    #text = textblob.correct()

    return text

In [9]:
# Testing
clean_comment("Yoooooo tihs stuff is       <p>fiiiireeeee</p> @Khoa")

'yo this stuff is fire'

In [10]:
TestDF['commentBody'] = TestDF['commentBody'].apply(clean_comment)
TestDF.to_csv("Data/DataTestComment.csv", index=False, mode="w+")

In [11]:
df['commentBody'] = df['commentBody'].apply(clean_comment)
df.to_csv("Data/Cleaned.csv", index=False, mode="w+")